In [33]:
# git config --global credential.helper store

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import itertools
from collections import Counter
import ssl
import time
import string
import unicodedata
%matplotlib inline

from urllib.request import Request, urlopen
from threading import Thread
from bs4 import BeautifulSoup

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier, RandomForestRegressor,
                            GradientBoostingClassifier, GradientBoostingRegressor)
from sklearn.naive_bayes import MultinomialNB

import nltk
nltk.download(["stopwords", "punkt", "averaged_perceptron_tagger", "maxent_treebank_pos_tagger"])
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk import RegexpParser
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

from clean_dfs import clean_features, clean_weeks, clean_lyrics
from web_scraping import parse_page, store_lyrics
from nlp_pipeline import lyrics_tokenize, get_tfidf_matrix
from genre_helper_functions import get_bucket, contains_genre_type, create_genre_column
from make_plots import (make_frequency_plot, make_line_plot, make_dual_plot_same,
                        make_dual_plot_mixed, make_scatter)
import modeling_functions as mf

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


In [2]:
features = clean_features()
#weeks = clean_weeks()

In [3]:
'''joined = weeks.merge(features, on='SongID')
#joined.to_csv("data/joined.csv", index=False)

# Expand genres into individual components
featureGenres = features.explode('spotify_genre')
featureGenres = featureGenres[featureGenres['spotify_genre'] != '']

joinedGenres = joined.explode('spotify_genre')
joinedGenres = joinedGenres[joinedGenres['spotify_genre'] != '']

explicitness = joined[['Year', 'spotify_track_explicit']]
explicitness = explicitness.groupby(['Year']).mean().reset_index()

numericalMetrics = joined.columns.tolist()[11:23]
numericals = joined[['Year'] + numericalMetrics].groupby(['Year']).mean().reset_index()'''

'joined = weeks.merge(features, on=\'SongID\')\n#joined.to_csv("data/joined.csv", index=False)\n\n# Expand genres into individual components\nfeatureGenres = features.explode(\'spotify_genre\')\nfeatureGenres = featureGenres[featureGenres[\'spotify_genre\'] != \'\']\n\njoinedGenres = joined.explode(\'spotify_genre\')\njoinedGenres = joinedGenres[joinedGenres[\'spotify_genre\'] != \'\']\n\nexplicitness = joined[[\'Year\', \'spotify_track_explicit\']]\nexplicitness = explicitness.groupby([\'Year\']).mean().reset_index()\n\nnumericalMetrics = joined.columns.tolist()[11:23]\nnumericals = joined[[\'Year\'] + numericalMetrics].groupby([\'Year\']).mean().reset_index()'

In [4]:
'''# Normalize numerical features not between 0 and 1
featureGenresNorm = featureGenres.copy()
scaled = ["track_duration", "loudness", "tempo"]
for metric in scaled:
    mms = MinMaxScaler()
    featureGenresNorm[metric] = mms.fit_transform(featureGenresNorm['track_duration']. \
                                to_numpy().reshape(-1, 1))

# Create grouped tables
genres = featureGenres.groupby(['spotify_genre'])['SongID'].count().reset_index()
genresJoined = joinedGenres.groupby(['spotify_genre'])['SongID'].count().reset_index()
genresJoinedDecade = joinedGenres.groupby(['spotify_genre', 'Decade'])['SongID'].count(). \
                        reset_index().sort_values(by="Decade")
genreFeatures = featureGenresNorm.groupby(['spotify_genre'])[numericalMetrics].mean().reset_index()'''

'# Normalize numerical features not between 0 and 1\nfeatureGenresNorm = featureGenres.copy()\nscaled = ["track_duration", "loudness", "tempo"]\nfor metric in scaled:\n    mms = MinMaxScaler()\n    featureGenresNorm[metric] = mms.fit_transform(featureGenresNorm[\'track_duration\'].                                 to_numpy().reshape(-1, 1))\n\n# Create grouped tables\ngenres = featureGenres.groupby([\'spotify_genre\'])[\'SongID\'].count().reset_index()\ngenresJoined = joinedGenres.groupby([\'spotify_genre\'])[\'SongID\'].count().reset_index()\ngenresJoinedDecade = joinedGenres.groupby([\'spotify_genre\', \'Decade\'])[\'SongID\'].count().                         reset_index().sort_values(by="Decade")\ngenreFeatures = featureGenresNorm.groupby([\'spotify_genre\'])[numericalMetrics].mean().reset_index()'

In [5]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [6]:
'''# Web scrape lyrics
featureScrape = features.loc[[contains_genre_type(genre, ["pop", "rock", "metal"]) for genre \
                             in features['spotify_genre']]].reset_index(drop=True)
lyricsMap = {}
threads = []
temp = 0
start = time.time()
# Write scraped lyrics to hashmap, parallelize to save time (thread safe because no unique keys)
#for i in range(temp, temp+50):
for i in range(len(featureScrape)):
    t = Thread(target=store_lyrics, args=(featureScrape['Song'][i],
                featureScrape['Performer'][i], lyricsMap))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
end = time.time()
print(end - start)
scrapedLyrics = pd.DataFrame(lyricsMap.items(), columns=["SongID", "Lyrics"])
scrapedLyrics.to_csv("data/scrapedLyrics.csv", index=False)

# Get list of all improperly formatted songs and save to file
problemSongs = []
for k, v in lyricsMap.items():
    if v[0][0] == "*":
        problemSongs.append([k] + v[2:5])
print(len(featureScrape), len(problemSongs))

with open("data/problemSongs.txt", "w") as file:
    for s in problemSongs:
        file.write("{}\n".format(s))'''

'# Web scrape lyrics\nfeatureScrape = features.loc[[contains_genre_type(genre, ["pop", "rock", "metal"]) for genre                              in features[\'spotify_genre\']]].reset_index(drop=True)\nlyricsMap = {}\nthreads = []\ntemp = 0\nstart = time.time()\n# Write scraped lyrics to hashmap, parallelize to save time (thread safe because no unique keys)\n#for i in range(temp, temp+50):\nfor i in range(len(featureScrape)):\n    t = Thread(target=store_lyrics, args=(featureScrape[\'Song\'][i],\n                featureScrape[\'Performer\'][i], lyricsMap))\n    threads.append(t)\n    t.start()\nfor t in threads:\n    t.join()\nend = time.time()\nprint(end - start)\nscrapedLyrics = pd.DataFrame(lyricsMap.items(), columns=["SongID", "Lyrics"])\nscrapedLyrics.to_csv("data/scrapedLyrics.csv", index=False)\n\n# Get list of all improperly formatted songs and save to file\nproblemSongs = []\nfor k, v in lyricsMap.items():\n    if v[0][0] == "*":\n        problemSongs.append([k] + v[2:5])\nprin

In [7]:
# Read csv of previously outputted scraped lyrics and reformat to match original
allLyrics = clean_lyrics()

In [8]:
'''# NLP pipeline to create tokens from lyrics
allLyrics['Lyrics_tokenized'] = list(map(lyrics_tokenize, allLyrics['Lyrics']))
allLyrics.dropna(inplace=True)
allLyrics.to_csv("data/lyricsTokenized.csv", index=False)'''

'# NLP pipeline to create tokens from lyrics\nallLyrics[\'Lyrics_tokenized\'] = list(map(lyrics_tokenize, allLyrics[\'Lyrics\']))\nallLyrics.dropna(inplace=True)\nallLyrics.to_csv("data/lyricsTokenized.csv", index=False)'

In [9]:
###################

In [10]:
# Create corpus and make dataframe with TF-IDF matrix
allLyrics = pd.read_csv("data/lyricsTokenized.csv")
allLyrics.dropna(inplace=True)
corpus = allLyrics['Lyrics_tokenized']
tfidfLyrics = get_tfidf_matrix(corpus, 5000)
tfidfLyrics.insert(0, "SongID", allLyrics['SongID'])
tfidfLyrics.to_csv("data/tfidfMatrix.csv", index=False)

In [68]:
# Join with features to get valence of songs with lyrics
valenceOnly = pd.DataFrame({"SongID": features['SongID'], "spotify_genre": features['spotify_genre'], "valence": features['valence']})
lyricsAndValence = tfidfLyrics.merge(valenceOnly, on='SongID')
lyricsAndValence.set_index("SongID", inplace=True)
# Temporary command to change model from regressor to classifier
lyricsAndValence['valence'] = (lyricsAndValence['valence'] > 0.5).astype(int)

In [70]:
# Run models for pop genre
lyricsAndValencePop = lyricsAndValence[[contains_genre_type(g, ["pop"]) for g in lyricsAndValence['spotify_genre']]]
lyricsAndValencePop.drop(["spotify_genre"], axis=1, inplace=True)
X = lyricsAndValencePop[lyricsAndValencePop.columns.difference(['valence'])]
y = lyricsAndValencePop['valence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [71]:
# Logistic regression model
y_pred, logistic_regression_results = mf.get_logistic_regression_results(X_train, \
                                            X_test, y_train, y_test)
print(logistic_regression_results)
# 0.5559, 0.3957, 0.3917

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.4955957348168753, 0.5014031805425632, 0.4912923923006416)


In [42]:
# Gradient boosting model
start = time.time()
mf.plot_gradient_boost_class_hyperparameters(X_train, X_test, y_train, y_test, \
                                             "pop")
end = time.time()
print(end-start)

KeyboardInterrupt: 

In [72]:
start = time.time()
y_pred, gradient_boost_class_results = mf.get_gradient_boost_class_results(0.2, 150, \
                                            1.0, 5000, X_train, X_test, y_train, y_test)
print(gradient_boost_class_results)
end = time.time()
print(end-start)
# 0.6 0.6 0.6

(0.5280482151135837, 0.46117867165575305, 0.5272727272727272)
77.58851027488708


6.804572672836147e-05


0.6361594991834513

14696

SongID
The ArcherTaylor Swift                                         0
TakeawayThe Chainsmokers & Illenium Featuring Lennon Stella    0
Bad Bad BadYoung Thug Featuring Lil Baby                       1
HeartlessThe Weeknd                                            0
HeatChris Brown Featuring Gunna                                0
Yellow HeartsAnt Saunders                                      1
Don't Check On MeChris Brown Featuring Justin Bieber & Ink     1
Cornelia StreetTaylor Swift                                    0
GoldenHarry Styles                                             0
Watermelon SugarHarry Styles                                   0
Easier5 Seconds Of Summer                                      1
TrampolineSHAED                                                0
LivingDierks Bentley                                           0
Prayed For YouMatt Stell                                       0
When I Grow UpNF                                               0
Break Up With Your